In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
from scipy import interpolate
from datetime import datetime
import datetime as datetime_super


def read_csv():
    with open("power_generation.csv") as csvfile:
        datareader = csv.reader(csvfile, delimiter=',')
        timestamps = []
        features = []
        num_features = len(next(datareader)) - 1
        csvfile.seek(0)
        next(datareader, None)

        for i in range(num_features):
            features.append([])
        for row in datareader:
            timestamps.append(row[0])
            for i in range(num_features):
                features[i].append(float(row[i+1]))

        returned_values = []
        returned_values.append(timestamps)
        for i in range(num_features):
            returned_values.append(features[i])

    return returned_values


def prepare_dataset(df_timestamps):
    for i in range(len(df_timestamps["Timestamp"])):
        value = int(df_timestamps["Timestamp"][i][11:13])
        value -= 1
        value = "{:02d}".format(value)
        value = str(value)
        value = df_timestamps["Timestamp"][i][0:11] + value
        df_timestamps["Timestamp"][i] = value

    df_timestamps["Timestamp"] = pd.to_datetime(df_timestamps["Timestamp"])

    return df_timestamps


def revert_dataset(df):
    for i in range(len(df["Timestamp"])):
        value = int(df["Timestamp"][i][11:13])
        value += 1
        value = "{:02d}".format(value)
        value = str(value)
        value = df["Timestamp"][i][0:11] + value
        df["Timestamp"][i] = value

    return df


def add_timestamps():
    start = datetime.strptime("2019.09.23.00", "%Y.%m.%d.%H")
    end = datetime.strptime("2019.10.04.23", "%Y.%m.%d.%H")
    date_generated = [start + datetime_super.timedelta(hours=x) for x in range(0, (end-start).days*24)]
    timestamp_list_str = []
    for date in date_generated:
        timestamp_list_str.append(date.strftime("%Y.%m.%d.%H"))

    start = datetime.strptime("2019.11.14.00", "%Y.%m.%d.%H")
    end = datetime.strptime("2019.11.15.00", "%Y.%m.%d.%H")
    date_generated = [start + datetime_super.timedelta(hours=x) for x in range(0, (end-start).days*24)]
    for date in date_generated:
        timestamp_list_str.append(date.strftime("%Y.%m.%d.%H"))

    start = datetime.strptime("2019.12.12.00", "%Y.%m.%d.%H")
    end = datetime.strptime("2019.12.13.00", "%Y.%m.%d.%H")
    date_generated = [start + datetime_super.timedelta(hours=x) for x in range(0, (end-start).days*24)]
    for date in date_generated:
        timestamp_list_str.append(date.strftime("%Y.%m.%d.%H"))

    start = datetime.strptime("2020.03.12.00", "%Y.%m.%d.%H")
    end = datetime.strptime("2020.03.13.00", "%Y.%m.%d.%H")
    date_generated = [start + datetime_super.timedelta(hours=x) for x in range(0, (end-start).days*24)]
    for date in date_generated:
        timestamp_list_str.append(date.strftime("%Y.%m.%d.%H"))

    return timestamp_list_str


def predict_value(df_timestamps, feature, prediction):
    skip = 8160 + 2880
    num_points = skip + 5000
    xpoints = df_timestamps[skip:num_points].to_numpy()[:, 0].astype(float)
    ypoints = np.array(feature[skip:num_points])

    x = [prediction]
    tck = interpolate.splrep(xpoints, ypoints)

    return interpolate.splev(x, tck)


def print_missing_data(df_timestamps, row_index, col_index, predictions, feature, timestamp):
    predictions.loc[row_index, col_index] = predict_value(df_timestamps, feature, timestamp)

    return predictions


values = read_csv()
timestamps = values[0]
df_timestamps = pd.DataFrame(np.array(timestamps).reshape(len(timestamps), 1), columns = ["Timestamp"])
df_timestamps_original = df_timestamps.copy()

geothermal = values[1]
biomass = values[2]
biogas = values[3]
small_hydro = values[4]
wind_total = values[5]
solar_pv = values[6]
solar_thermal = values[7]
renewables = values[8]
nuclear = values[9]
thermal = values[10]
imports = values[11]
hydro = values[12]

feature_list = ["Timestamp","Geothermal","Biomass","Biogas","Small Hydro","Wind Total","Solar PV","Solar Thermal","Renewables","Nuclear","Thermal","Imports","Hydro"]

df_original = df_timestamps.copy()
i = 0
for feature in values:
    if(feature != values[0]):
        df_original[feature_list[i]] = np.array(feature).reshape(len(feature))
    i += 1

df_timestamps = prepare_dataset(df_timestamps)

timestamp_list_str = add_timestamps()

df_timestamps_predictions = pd.DataFrame(np.asarray(timestamp_list_str).reshape(len(timestamp_list_str), 1), columns = ["Timestamp"])
df_timestamps_predictions_str = df_timestamps_predictions.copy()
df_timestamps_predictions_str = revert_dataset(df_timestamps_predictions_str)
df_timestamps_predictions["Timestamp"] = pd.to_datetime(df_timestamps_predictions["Timestamp"])
timestamp_list = df_timestamps_predictions.to_numpy().flatten().tolist()

df_predictions = pd.DataFrame(columns=feature_list)
df_predictions["Timestamp"] = df_timestamps_predictions["Timestamp"]
row_index = 0
col_index = 0
for timestamp in timestamp_list:
    col_index = 0
    for feature in values:
        if(feature != values[0]):
            df_predictions = print_missing_data(df_timestamps, row_index, feature_list[col_index], df_predictions, feature, timestamp)
        col_index += 1
    row_index += 1

df_timestamps = df_timestamps_original.append(df_timestamps_predictions_str)
df_timestamps.sort_values("Timestamp", inplace=True)

df_original = prepare_dataset(df_original)

df = df_original.append(df_predictions)
df.sort_values("Timestamp", inplace=True)
df["Timestamp"] = df_timestamps["Timestamp"]

for col in df.columns:
    if(col != "Timestamp"):
        df[col] = df[col].apply(lambda x : x if x > 0 else 0) # replace negative numbers with 0

df.to_csv("power_generation_processed.csv", index=False)


C:\Users\shour\AppData\Local\Temp/ipykernel_21896/2905844775.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_timestamps["Timestamp"][i] = value
